## Make regulariztion

通过将已经打好标签的训练文件夹里的图片描述信息提取出来，然后去掉要训练的词汇，使用其它词汇来生成对比图片，以更好的训练模型。

In [13]:
import json
from urllib import request, parse
import random
import uuid
import urllib.request
import urllib.parse
import io
import os
import time
from PIL.PngImagePlugin import PngInfo
import hashlib
from urllib.error import URLError


In [14]:
from enum import Enum, unique

@unique
class RunMode(Enum):
    LOCAL = 1
    POD_LOCAL = 2
    POD_REMOTE = 3

run_mode = RunMode.LOCAL
# 设置运行模式，来实现在本机，runpod本地，runpod远程的切换
server_address = "http://yc7sxv2vriceb8-3000.proxy.runpod.net" if run_mode == RunMode.POD_REMOTE else "0.0.0.0:3000" if run_mode == RunMode.POD_LOCAL else "127.0.0.1:8188"
client_id = str(uuid.uuid4())


In [15]:
def changeImageSize(width, height):
    # 最小的宽高为本机896，runpod1024
    limited = 896 if run_mode == RunMode.LOCAL else 1024
    minSize = min(width, height)

    # 缩放宽高，使得最窄的边等于limited
    if minSize > limited:
        scale = minSize / limited
        widthScale = int(width / scale)
        heightScale = int(height / scale)
    else:
        scale = limited / minSize
        widthScale = int(width * scale)
        heightScale = int(height * scale)

    # 将两个值都除以64，然后取整,也就是说边长必须是64的倍数
    widthScale = int(widthScale/64) * 64
    heightScale = int(heightScale/64) * 64

    return widthScale, heightScale

def checkImageExsit(imageDir, imageName):
    for file in os.listdir(imageDir):
        if file.startswith(imageName):
            return True
    return False


def queue_prompt(prompt):
    while True:
        try:
            p = {"prompt": prompt, "client_id": client_id}
            data = json.dumps(p).encode('utf-8')
            req = urllib.request.Request("http://{}/prompt".format(server_address), data=data)
            return json.loads(urllib.request.urlopen(req, timeout=5).read())
        except URLError:
            print("连接超时，正在重试...")

def get_history(prompt_id):
    with urllib.request.urlopen("http://{}/history/{}".format(server_address, prompt_id)) as response:
        return json.loads(response.read())

def get_image(filename, subfolder, folder_type):
    data = {"filename": filename, "subfolder": subfolder, "type": folder_type}
    url_values = urllib.parse.urlencode(data)
    with urllib.request.urlopen("http://{}/view?{}".format(server_address, url_values)) as response:
        return response.read()

def handle_whitespace(string: str):
    return string.strip().replace("\n", " ").replace("\r", " ").replace("\t", " ")

def parse_name(ckpt_name):
    path = ckpt_name
    filename = path.split("/")[-1]
    filename = filename.split(".")[:-1]
    filename = ".".join(filename)
    return filename


def calculate_sha256(file_path):
    sha256_hash = hashlib.sha256()

    with open(file_path, "rb") as f:
        # Read the file in chunks to avoid loading the entire file into memory
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)

    return sha256_hash.hexdigest()

def get_images(ws, prompt):
    prompt_id = queue_prompt(prompt)['prompt_id']
    output_images = {}
    while True:
        out = ws.recv()
        if isinstance(out, str):
            message = json.loads(out)
            if message['type'] == 'executing':
                data = message['data']
                if data['node'] is None and data['prompt_id'] == prompt_id:
                    break #Execution is done
        else:
            continue #previews are binary data

    history = get_history(prompt_id)[prompt_id]
    for o in history['outputs']:
        for node_id in history['outputs']:
            node_output = history['outputs'][node_id]
            if 'images' in node_output:
                images_output = []
                for image in node_output['images']:
                    image_data = get_image(image['filename'], image['subfolder'], image['type'])
                    images_output.append(image_data)
                output_images[node_id] = images_output

    return output_images
    
def saveImages(result, imageDir,imageName,comment,previewImage=0): 
    metadata = PngInfo()
    metadata.add_text("parameters", comment)  
    k = 0 
    for node_id in result:    
        if k < previewImage:
            k += 1
            continue    
        for image_data in result[node_id]:            
            image = Image.open(io.BytesIO(image_data))
            imagePath = imageDir + "/" + imageName + f"_{k}" + ".png"
            image.save(imagePath,pnginfo=metadata) 
            k += 1
    

## import json file

In [16]:
promptJson = 'api.json'

with open(promptJson) as f:
    prompt = json.load(f)

In [17]:
# 训练文件夹名称
trainingFolder = 'c:/Users/BigHippo78/Pictures/cosplay_training/img'

# 生成图片文件夹
imageDir = 'c:/Users/BigHippo78/Pictures/cosplay_training/reg'

# 如果文件夹不存在，创建文件夹
if not os.path.exists(imageDir):
    os.mkdir(imageDir)


In [18]:

from PIL import Image
from sdparsers import ParserManager
import websocket

ws = websocket.WebSocket()
ws.connect("ws://{}/ws?clientId={}".format(server_address, client_id))

batch_size = 4

# 提取训练文件夹下所有的txt文件
prompt_list = []
for file in os.listdir(trainingFolder):
    if file.endswith(".txt"):
        # 读取txt文件内容
        with open(os.path.join(trainingFolder, file), 'r', encoding='utf-8') as f:
            prompt_list.append(f.read())

# 遍历提示词组
for i in range(len(prompt_list)):
        input_text = prompt_list[i]      
        # 去掉提示词中的训练词 
        prompt['6']['inputs']['text'] = input_text       
        
        # 设置生成图片的名字模板
        originalImagName = f"reg_{i}"
        # 检测目标文件夹里是否已经存在该图片，如果存在，跳过
        if checkImageExsit(imageDir, originalImagName):
            print(f"{originalImagName}已经存在，跳过")
            continue   
        
        # 下面是确定实际生成图片的大小，种子，以及保存图片信息
        # 保存图片的信息使用的是38号节点，也就是Save Image w/Metadata节点 
        prompt['5']['inputs']['batch_size'] = batch_size        
        
        seed = random.randint(0, 976242998978323)
        prompt['3']['inputs']['seed'] = seed        
        
        
        # 连接api，生成并保存图片
        start_time = time.time()
        
        images = get_images(ws, prompt)   
        saveImages(images, imageDir, originalImagName, comment='comment',previewImage=0)     
        end_time = time.time()
        
        # 生成batch_size个文本文件，把input_text写入文本文件
        for j in range(batch_size):
            with open(os.path.join(imageDir, f"{originalImagName}_{j}.txt"), 'w', encoding='utf-8') as f:
                f.write(input_text)
        print(f"已经完成第{i}张图片，耗时{end_time-start_time}秒")       
        
            

reg_0已经存在，跳过
reg_1已经存在，跳过
reg_2已经存在，跳过
reg_3已经存在，跳过
reg_4已经存在，跳过
reg_5已经存在，跳过
reg_6已经存在，跳过
reg_7已经存在，跳过
reg_8已经存在，跳过
已经完成第9张图片，耗时120.860515832901秒
已经完成第10张图片，耗时119.36887836456299秒
已经完成第11张图片，耗时121.01859855651855秒
已经完成第12张图片，耗时118.83589172363281秒
已经完成第13张图片，耗时119.92179346084595秒
已经完成第14张图片，耗时118.67353296279907秒
已经完成第15张图片，耗时118.87743711471558秒
已经完成第16张图片，耗时119.56634330749512秒
已经完成第17张图片，耗时118.34068202972412秒
已经完成第18张图片，耗时121.73725581169128秒
已经完成第19张图片，耗时119.81167912483215秒
已经完成第20张图片，耗时118.85611343383789秒
已经完成第21张图片，耗时119.50131392478943秒
已经完成第22张图片，耗时118.77938270568848秒
已经完成第23张图片，耗时121.62491703033447秒
已经完成第24张图片，耗时118.02148795127869秒
已经完成第25张图片，耗时118.8294575214386秒
已经完成第26张图片，耗时119.64476609230042秒
已经完成第27张图片，耗时118.33725380897522秒
已经完成第28张图片，耗时123.7270827293396秒
已经完成第29张图片，耗时119.47732925415039秒
已经完成第30张图片，耗时119.38465547561646秒
已经完成第31张图片，耗时122.1316876411438秒
已经完成第32张图片，耗时118.53812718391418秒
已经完成第33张图片，耗时121.34753680229187秒
已经完成第34张图片，耗时118.33962535858154秒
已经完成第35张图片，耗时118.7144410610199秒

In [19]:
import pygame
pygame.init()
my_sound = pygame.mixer.Sound('../../assets/sound/download-complete.wav')
my_sound.play()

pygame 2.5.2 (SDL 2.28.3, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html
